In [1]:
pip install gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [2]:
pip install gradio

In [4]:
import requests
import json
from gtts import gTTS
import gradio as gr
import tempfile

# Replace with your actual API key
OPENROUTER_API_KEY = "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"

def generate_story_with_audio(language_choice, title):
    # Language settings
    lang_map = {
        "Hindi": ("हिन्दी", "hi", f"'{title}' शीर्षक पर एक छोटी और भावनात्मक कहानी बनाएं हिन्दी में, जो बच्चों के लिए हो, और जिसमें एक नैतिक शिक्षा हो।"),
        "Marathi": ("मराठी", "mr", f"'{title}' या शीर्षकावर एक छोटी आणि भावनिक कथा तयार करा मराठीत, जी मुलांसाठी असेल आणि ज्यामध्ये एक नैतिक शिकवण असेल.")
    }

    if language_choice not in lang_map:
        return "Invalid language", None

    lang_name, lang_code, prompt = lang_map[language_choice]

    # Generate story from OpenRouter
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {OPENROUTER_API_KEY}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "deepseek/deepseek-r1:free",
            "messages": [{"role": "user", "content": prompt}]
        })
    )

    if response.status_code != 200:
        return f"Error: {response.status_code}", None

    story = response.json()["choices"][0]["message"]["content"]

    # Convert story to speech
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    gTTS(text=story, lang=lang_code).save(tmp_file.name)

    return story, tmp_file.name


# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 📖 Short Moral Story Generator with Audio")

    language = gr.Dropdown(choices=["Hindi", "Marathi"], label="Choose Language", value="Hindi")
    title_input = gr.Textbox(label="Story Title", placeholder="Enter your story title here...")
    generate_btn = gr.Button("Generate Story")

    story_output = gr.Textbox(label="Generated Story", lines=10)
    audio_output = gr.Audio(label="Story Audio", type="filepath")

    generate_btn.click(
        fn=generate_story_with_audio,
        inputs=[language, title_input],
        outputs=[story_output, audio_output]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c12991408de0806cb7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
